In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('recommendation').getOrCreate()

In [2]:
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator

In [3]:
df_movie = spark.read.csv('../Arquivos/movielens_ratings.csv', inferSchema=True, header=True)
df_movie.printSchema()

root
 |-- movieId: integer (nullable = true)
 |-- rating: double (nullable = true)
 |-- userId: integer (nullable = true)



In [4]:
df_movie.show()

+-------+------+------+
|movieId|rating|userId|
+-------+------+------+
|      2|   3.0|     0|
|      3|   1.0|     0|
|      5|   2.0|     0|
|      9|   4.0|     0|
|     11|   1.0|     0|
|     12|   2.0|     0|
|     15|   1.0|     0|
|     17|   1.0|     0|
|     19|   1.0|     0|
|     21|   1.0|     0|
|     23|   1.0|     0|
|     26|   3.0|     0|
|     27|   1.0|     0|
|     28|   1.0|     0|
|     29|   1.0|     0|
|     30|   1.0|     0|
|     31|   1.0|     0|
|     34|   1.0|     0|
|     37|   1.0|     0|
|     41|   2.0|     0|
+-------+------+------+
only showing top 20 rows



In [5]:
df_movie.describe().show()

+-------+------------------+------------------+------------------+
|summary|           movieId|            rating|            userId|
+-------+------------------+------------------+------------------+
|  count|              1501|              1501|              1501|
|   mean| 49.40572951365756|1.7741505662891406|14.383744170552964|
| stddev|28.937034065088994| 1.187276166124803| 8.591040424293272|
|    min|                 0|               1.0|                 0|
|    max|                99|               5.0|                29|
+-------+------------------+------------------+------------------+



In [6]:
train, test = df_movie.randomSplit([0.8, 0.2])

In [7]:
als = ALS(maxIter=5, regParam=0.01, userCol='userId', itemCol='movieId', ratingCol='rating')

In [8]:
model = als.fit(train)

In [9]:
predictions = model.transform(test)

In [10]:
predictions.show()

+-------+------+------+-----------+
|movieId|rating|userId| prediction|
+-------+------+------+-----------+
|      1|   1.0|     5|-0.28109938|
|      1|   1.0|     6|  1.0581338|
|      6|   1.0|     6|   1.029419|
|      3|   1.0|     7|  2.2523906|
|      3|   1.0|    13|  1.2729951|
|      3|   1.0|    21|  2.2132719|
|      5|   1.0|     9|    2.08036|
|      5|   2.0|    18|  1.2379119|
|      9|   1.0|     3|  0.5758105|
|      9|   1.0|     8| 0.90769887|
|      4|   3.0|     2| -0.6432719|
|      4|   3.0|    10|-0.17141086|
|      4|   1.0|    24|  0.9776964|
|      8|   1.0|    10|  3.2176356|
|      8|   1.0|    20|  2.4225054|
|      7|   1.0|    10|  0.7941544|
|      7|   1.0|    16|  2.7411978|
|      7|   1.0|    24|  3.1259353|
|      2|   1.0|    26|  1.5852787|
|      0|   1.0|    13| 0.36981943|
+-------+------+------+-----------+
only showing top 20 rows



In [11]:
evaluator = RegressionEvaluator(metricName='rmse', labelCol='rating', predictionCol='prediction')

In [12]:
rmse = evaluator.evaluate(predictions)

In [13]:
print(f"RMSE: {rmse}")

RMSE: 1.961068440335141


In [14]:
single_user = test.filter(test['userId'] == 11).select(['movieId', 'userId'])

In [15]:
single_user.show()

+-------+------+
|movieId|userId|
+-------+------+
|     13|    11|
|     18|    11|
|     38|    11|
|     40|    11|
|     41|    11|
|     43|    11|
|     50|    11|
|     62|    11|
|     64|    11|
|     67|    11|
|     72|    11|
+-------+------+



In [16]:
recommendations = model.transform(single_user)

In [17]:
recommendations.orderBy('prediction', ascending=False).show()

+-------+------+----------+
|movieId|userId|prediction|
+-------+------+----------+
|     38|    11| 3.8804674|
|     13|    11| 3.4680274|
|     50|    11|  3.268357|
|     64|    11| 2.4046354|
|     18|    11| 1.7370473|
|     72|    11|0.64507127|
|     67|    11|0.18544936|
|     40|    11|0.07191455|
|     43|    11|0.06572983|
|     41|    11|-1.0054898|
|     62|    11|-2.4951513|
+-------+------+----------+

